In [5]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
import statsmodels.api as sm
np.random.seed(0)


In [13]:
def loglikelihood_normal(mu, sigma2, y):
    y = np.asanyarray(y)
    n = len(y)
    return -0.5 * n * np.log(2 * np.pi * sigma2) - np.sum((y - mu)**2) / (2 * sigma2)

In [14]:
def mle_unrestricted(y):
    y = np.asarray(y)
    mu_hat = np.mean(y)
    sigma2_hat = np.mean((y - mu_hat)**2)
    return float(mu_hat), float(sigma2_hat)

def mle_null(y, mu0):
    y = np.asarray(y)
    mu_hat0 = mu0
    sigma2_hat0 = np.mean((y - mu0)**2)
    return float(mu_hat0), float(sigma2_hat0)

In [15]:
# sanity check
y_test = np.array([1.0, 2.0, 3.0, 4.0])
print(mle_unrestricted(y_test))
print(mle_null(y_test, mu0=0.0))

(2.5, 1.25)
(0.0, 7.5)


In [19]:
def lrt_stat_normal_unknown_sigma(y, mu0):
    y = np.asarray(y)

    mu_hat, sigma2_hat = mle_unrestricted(y)
    mu_hat0, sigma2_hat0 = mle_null(y, mu0)

    ll_full = loglikelihood_normal(mu_hat, sigma2_hat, y)
    ll_null = loglikelihood_normal(mu_hat0, sigma2_hat0, y)

    lam = np.exp(ll_null - ll_full)

    T = -2 * (ll_null - ll_full)

    details = {
        "mu_hat": float(mu_hat),
        "sigma2_hat": float(sigma2_hat),
        "mu_hat0": float(mu_hat0),
        "sigma2_hat0": float(sigma2_hat0),
        "ll_full": float(ll_full),
        "ll_null": float(ll_null)
    }

    return float(lam), float(T)

y_example = np.random.normal(loc=0.0, scale=1.0, size=20)
lam, T = lrt_stat_normal_unknown_sigma(y_example, mu0=0.0)
lam, T

(0.12712365500802003, 4.125190009487909)

In [27]:
def t_stat_normal_unknown_sigma(y, mu0):
    y = np.asarray(y)
    n = len(y)
    y_bar = np.mean(y)
    s2 = np.sum((y - y_bar)**2) / (n - 1)
    s = np.sqrt(s2)
    return (y_bar - mu0) / (s / np.sqrt(n))

In [21]:
def simulate_T_under_H0(mu0=0.0, sigma=1.0, n=20, reps=5000):
    Ts = []
    for _ in range(reps):
        y = np.random.normal(mu0, sigma, size=n)
        lam, T = lrt_stat_normal_unknown_sigma(y, mu0)
        Ts.append(T)
    return np.array(Ts)

In [23]:
Ts = simulate_T_under_H0()

float(Ts.mean()), float(Ts.var())

(1.0229662421679055, 2.037961804259129)

In [25]:
def lrt_test_normal_mean(y, mu0=0.0, alpha=0.05):
    lam, T = lrt_stat_normal_unknown_sigma(y, mu0)
    t = t_stat_normal_unknown_sigma(y, mu0)
    p = 1 - stats.t.cdf(t, df=len(y)-1)
    return p < alpha

In [29]:
y = np.array([5.2, 4.8, 6.1, 5.7, 4.9, 5.3])
reject = lrt_test_normal_mean(y, mu0=5.0, alpha=0.05)
reject

np.False_